# 1. Memoria con PostgreSQL
https://langchain-ai.github.io/langgraph/how-tos/persistence_postgres/

In [30]:
from decouple import config

DB_HOST = config("DB_HOST")
DB_PORT= config("DB_PORT")
DB_NAME= config("DB_NAME")
DB_USER= config("DB_USER")
DB_PASSWORD= config("DB_PASSWORD")

In [31]:
DB_URI = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}?sslmode=disable"

In [32]:
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}

## Define model and tools for the graph


In [33]:
from typing import Literal

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## Use sync connection


In [34]:
from psycopg_pool import ConnectionPool

with ConnectionPool(
    # Example configuration
    conninfo=DB_URI,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)

    # NOTE: you need to call .setup() the first time you're using your checkpointer
    checkpointer.setup()

    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "1"}}
    res = graph.invoke({"messages": [("human", "what's the weather in sf")]}, config)
    checkpoint = checkpointer.get(config)

In [35]:
res

{'messages': [HumanMessage(content="what's the weather in sf", additional_kwargs={}, response_metadata={}, id='d9a8e7db-0166-4b95-939f-ddebc4c78074'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KKcWgCvP3ILv7DU18dGRdfUv', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 57, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-31200184-08e2-4731-bf9e-c7be3cebac38-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_KKcWgCvP3ILv7DU18dGRdfUv', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, '

In [36]:
checkpoint

{'v': 1,
 'id': '1efcfda1-44ca-69e6-8012-0f89ad01f512',
 'ts': '2025-01-11T05:08:22.216285+00:00',
 'pending_sends': [],
 'versions_seen': {'agent': {'tools': '00000000000000000000000000000019.0.7194019897358124',
   'start:agent': '00000000000000000000000000000017.0.2592669383603905'},
  'tools': {'branch:agent:should_continue:tools': '00000000000000000000000000000018.0.8686643368804985'},
  '__input__': {},
  '__start__': {'__start__': '00000000000000000000000000000016.0.16288013815677638'}},
 'channel_versions': {'agent': '00000000000000000000000000000020.0.9516154216152956',
  'tools': '00000000000000000000000000000020.0.7580221657107787',
  'messages': '00000000000000000000000000000020.0.5200829379257798',
  '__start__': '00000000000000000000000000000017.0.14581068488974636',
  'start:agent': '00000000000000000000000000000018.0.8171452088729027',
  'branch:agent:should_continue:tools': '00000000000000000000000000000019.0.389313083226011'},
 'channel_values': {'agent': 'agent',
  '

## Use async connection

In [44]:
from psycopg_pool import AsyncConnectionPool

async with AsyncConnectionPool(
    # Example configuration
    conninfo=DB_URI,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = AsyncPostgresSaver(pool)

    # NOTE: you need to call .setup() the first time you're using your checkpointer
    await checkpointer.setup()

    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "8"}}
    res = await graph.ainvoke(
        {"messages": [("human", "what's the weather in nyc")]}, config
    )

    checkpoint = await checkpointer.aget(config)

In [45]:
res

{'messages': [HumanMessage(content="what's the weather in nyc", additional_kwargs={}, response_metadata={}, id='357752e6-ed47-4ed5-9006-ff8ccb0e36e9'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4OXh9ddkXe4XhnEnLxDMO390', 'function': {'arguments': '{"city":"nyc"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 58, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3a5be048-3713-4f00-a194-4f3b747e8aad-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'nyc'}, 'id': 'call_4OXh9ddkXe4XhnEnLxDMO390', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58

In [47]:
checkpoint

{'v': 1,
 'id': '1efcfeed-f67a-6194-8003-d0a575a29a90',
 'ts': '2025-01-11T07:37:12.905123+00:00',
 'pending_sends': [],
 'versions_seen': {'agent': {'tools': '00000000000000000000000000000004.0.4934158829986399',
   'start:agent': '00000000000000000000000000000002.0.3802465834231634'},
  'tools': {'branch:agent:should_continue:tools': '00000000000000000000000000000003.0.596339409483013'},
  '__input__': {},
  '__start__': {'__start__': '00000000000000000000000000000001.0.3689151538392933'}},
 'channel_versions': {'agent': '00000000000000000000000000000005.0.24301487924279597',
  'tools': '00000000000000000000000000000005.0.14344479701080493',
  'messages': '00000000000000000000000000000005.0.9197836540655787',
  '__start__': '00000000000000000000000000000002.0.35088172412270713',
  'start:agent': '00000000000000000000000000000003.0.19100607466503583',
  'branch:agent:should_continue:tools': '00000000000000000000000000000004.0.17745408896865644'},
 'channel_values': {'agent': 'agent',


# Pruebas extra

In [52]:
async with AsyncConnectionPool(
    # Example configuration
    conninfo=DB_URI,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = AsyncPostgresSaver(pool)

    config = {"configurable": {"thread_id": "8"}}

    checkpoints = []
    async for checkpoint in checkpointer.alist(config):
        checkpoints.append(checkpoint)

In [49]:
checkpoints

[CheckpointTuple(config={'configurable': {'thread_id': '8', 'checkpoint_ns': '', 'checkpoint_id': '1efcfeed-f67a-6194-8003-d0a575a29a90'}}, checkpoint={'v': 1, 'id': '1efcfeed-f67a-6194-8003-d0a575a29a90', 'ts': '2025-01-11T07:37:12.905123+00:00', 'pending_sends': [], 'versions_seen': {'agent': {'tools': '00000000000000000000000000000004.0.4934158829986399', 'start:agent': '00000000000000000000000000000002.0.3802465834231634'}, 'tools': {'branch:agent:should_continue:tools': '00000000000000000000000000000003.0.596339409483013'}, '__input__': {}, '__start__': {'__start__': '00000000000000000000000000000001.0.3689151538392933'}}, 'channel_versions': {'agent': '00000000000000000000000000000005.0.24301487924279597', 'tools': '00000000000000000000000000000005.0.14344479701080493', 'messages': '00000000000000000000000000000005.0.9197836540655787', '__start__': '00000000000000000000000000000002.0.35088172412270713', 'start:agent': '00000000000000000000000000000003.0.19100607466503583', 'branc

In [60]:
async with AsyncConnectionPool(
    conninfo=DB_URI,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = AsyncPostgresSaver(pool)

    config = {"configurable": {"thread_id": "8"}}

    checkpoint = await checkpointer.aget(config)

    if checkpoint:
        print("Checkpoint encontrado:", checkpoint)
    else:
        print("No se encontró ningún checkpoint.")


Checkpoint encontrado: {'v': 1, 'id': '1efcfeed-f67a-6194-8003-d0a575a29a90', 'ts': '2025-01-11T07:37:12.905123+00:00', 'pending_sends': [], 'versions_seen': {'agent': {'tools': '00000000000000000000000000000004.0.4934158829986399', 'start:agent': '00000000000000000000000000000002.0.3802465834231634'}, 'tools': {'branch:agent:should_continue:tools': '00000000000000000000000000000003.0.596339409483013'}, '__input__': {}, '__start__': {'__start__': '00000000000000000000000000000001.0.3689151538392933'}}, 'channel_versions': {'agent': '00000000000000000000000000000005.0.24301487924279597', 'tools': '00000000000000000000000000000005.0.14344479701080493', 'messages': '00000000000000000000000000000005.0.9197836540655787', '__start__': '00000000000000000000000000000002.0.35088172412270713', 'start:agent': '00000000000000000000000000000003.0.19100607466503583', 'branch:agent:should_continue:tools': '00000000000000000000000000000004.0.17745408896865644'}, 'channel_values': {'agent': 'agent', 'm

In [61]:
checkpoint

{'v': 1,
 'id': '1efcfeed-f67a-6194-8003-d0a575a29a90',
 'ts': '2025-01-11T07:37:12.905123+00:00',
 'pending_sends': [],
 'versions_seen': {'agent': {'tools': '00000000000000000000000000000004.0.4934158829986399',
   'start:agent': '00000000000000000000000000000002.0.3802465834231634'},
  'tools': {'branch:agent:should_continue:tools': '00000000000000000000000000000003.0.596339409483013'},
  '__input__': {},
  '__start__': {'__start__': '00000000000000000000000000000001.0.3689151538392933'}},
 'channel_versions': {'agent': '00000000000000000000000000000005.0.24301487924279597',
  'tools': '00000000000000000000000000000005.0.14344479701080493',
  'messages': '00000000000000000000000000000005.0.9197836540655787',
  '__start__': '00000000000000000000000000000002.0.35088172412270713',
  'start:agent': '00000000000000000000000000000003.0.19100607466503583',
  'branch:agent:should_continue:tools': '00000000000000000000000000000004.0.17745408896865644'},
 'channel_values': {'agent': 'agent',
